In [ ]:
import random
import numpy as np
from collections import defaultdict
import time
import dimod
import matplotlib.pyplot as plt

from tabu import TabuSampler
sampler_tabu = TabuSampler()


def compute_energy(spins, hamiltonian):
    """Compute the Hamiltonian energy for a given spin configuration."""
    energy = 0.0
    for term, weight in hamiltonian.items():
        if len(term) == 1:  # 1-body term
            energy += weight * spins[term[0]]
        elif len(term) == 2:  # 2-body term
            energy += weight * spins[term[0]] * spins[term[1]]
    return energy

def compute_energy_with_previous_info(spin, spin_chain, previous_energy, hamiltonian, nq):
    """Compute the Hamiltonian energy for a given spin configuration."""
    energy = previous_energy
    if (spin,) in hamiltonian.keys():
        energy += 2 * spin_chain[spin] * hamiltonian[(spin,)]
    list_neighbors = [k for k in hamiltonian.keys() if len(k) == 2 and spin in k]
    for edge in list_neighbors:
        energy += 2 * spin_chain[edge[0]] * spin_chain[edge[1]] * hamiltonian[edge] 
    return energy

def generate_neighbors(spins):
    """Generate neighboring solutions by flipping one spin at a time."""
    neighbors = []
    for spin in spins:
        new_spins = spins.copy()
        new_spins[spin] *= -1  # Flip spin
        neighbors.append(new_spins)
    return neighbors

def tabu_search(hamiltonian, nq, max_iterations=100, tabu_size=10):
    """Tabu Search to minimize the Hamiltonian energy."""
    # Initialize random spin configuration
    spins = {i: random.choice([-1, 1]) for i in range(nq)}
    best_solution = spins.copy()
    best_energy = compute_energy(spins, hamiltonian)
    Nevals = 1
    tabu_list = []
    
    for _ in range(max_iterations):
        neighbors = generate_neighbors(spins)
        best_candidate = None
        best_candidate_energy = float('inf')
        current_energy = compute_energy(spins, hamiltonian)
        Nevals += 1
        for ii, candidate in enumerate(neighbors):
            candidate_energy = compute_energy_with_previous_info(ii, candidate, current_energy, hamiltonian,nq)
            if candidate not in tabu_list and candidate_energy < best_candidate_energy:
                best_candidate = candidate
                best_candidate_energy = candidate_energy
        Nevals += 1
        if best_candidate is None:
            break  # No valid moves
        
        spins = best_candidate
        tabu_list.append(spins.copy())
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)  # Maintain tabu list size
        
        if best_candidate_energy < best_energy:
            best_solution = best_candidate.copy()
            best_energy = best_candidate_energy
    
    return best_solution, Nevals

def energy(x, hamiltonian):
    obj = 0
    spin = {"1":-1, "0":1}
    for k, v in hamiltonian.items():
        if len(k) == 2:
            obj += v * spin[x[k[0]]] * spin[x[k[1]]]
        elif len(k) == 1:
            obj += v * spin[x[k[0]]]
        else:
            print(k, v)
    return obj


def objective(samples_dict, hamiltonian, sol):
    optimal_energy = energy(sol, hamiltonian)
    results = []
    probability = 0
    for bitstring, counts in samples_dict.items():
        energy_sample = energy(bitstring, hamiltonian)
        r  = energy_sample/optimal_energy
        if np.abs(energy_sample - optimal_energy) < 1e-8:
            probability += counts
        if optimal_energy - energy_sample > 1e-10:
            probability += counts
            print(f"There is a better cost than the given by {optimal_energy - energy_sample}!")
            optimal_energy = energy_sample
        results.append([energy_sample, r, counts])
    results = np.array(results)
    shots = np.sum(results[:,2])
    mean_r = np.sum(results[:,0] * results[:,2])/(shots*optimal_energy)
    probability /= shots
    return {"results":np.array(results), "optimal_cost":optimal_energy, "r":mean_r, "probability":probability}

def objective_no_sol(samples_dict, hamiltonian):
    results = []
    probability = 0
    min_energy = 0
    for bitstring, counts in samples_dict.items():
        energy_sample = round(energy(bitstring, hamiltonian),6)
        if energy_sample < min_energy:
            min_energy = energy_sample
            probability = counts
            sol = bitstring
        elif energy_sample == min_energy:
            probability += counts
        results.append([energy_sample, counts])
    results = np.array(results)
    shots = np.sum(results[:,1])
    probability /= shots
    return {"results":np.array(results), "min_cost":min_energy, "probability":probability, "sol":sol}


def customized_running_tabu(hamiltonian, shots, sol, max_iterations, tabu_size=10):
    nq = len(sol)
    samples = defaultdict(int)
    ti = time.time()
    for _ in range(shots):
        final_spins, nevals = tabu_search(hamiltonian, nq, max_iterations, tabu_size)
        s = {-1:"0", 1:"1"}
        samples["".join(s[final_spins[i]] for i in range(nq))] += 1
    tf = time.time() - ti
    if sol:
        results = objective(samples, hamiltonian, sol)
    else:
        results = objective_no_sol(samples, hamiltonian)
    pd = 0.99
    results["nevals"] = nevals
    results["probability"] = results["probability"]
    results["STS"] = max(np.ceil(np.log(1 - pd) / np.log(1 - results["probability"])),1) if results["probability"] < 1 else 1
    results["time"] = tf
    return results

def ising_from_weights_terms(problem):
    h = {}
    J = {}
    for term, weight in problem.items():
        if len(term) == 1:
            h[term[0]] = weight
        else:
            J[term[0],term[1]] = weight
    return h, J

def running_tabu(hamiltonian, shots, sol, num_sweeps,list_size=20):
    h , J =  ising_from_weights_terms(hamiltonian) 
    bqm = dimod.BQM.from_ising(h, J)
    ti = time.time()
    r = sampler_tabu.sample(bqm, z,tenure=list_size)
    tf = time.time() - ti
    df = r.to_pandas_dataframe().sort_values('energy').reset_index(drop=True)
    samples = defaultdict(int)
    for _, row in df.iterrows():
        s_d = ''.join('0' if row[q] == 1 else '1' for q in row.keys() if type(q) == int)
        samples[s_d] += row["num_occurrences"]
    if sol:
        results = objective(samples, hamiltonian, sol)
    else:
        results = objective_no_sol(samples, hamiltonian)
    pd = 0.99
    results["probability"] = results["probability"]
    results["STS"] = max(np.ceil(np.log(1 - pd) / np.log(1 - results["probability"])),1) if results["probability"] < 1 else 1
    results["Nevals"] = results["STS"] * num_sweeps
    results["time"] = tf
    return results

def random_samples(nq, shots):
    samples = defaultdict(int)
    for i in range(shots):
        samples["".join(np.random.choice(["0","1"]) for _ in range(nq))] +=1
    return samples

In [30]:
problems = np.load(f"./Data/WMaxCut_cte_no_mdl.npy", allow_pickle=True).item()
nq = 40
kk = 0
H = problems[nq][kk]["ising_hamiltonian"]
sol = problems[nq][kk]["sol"]
p = 5
hamiltonian = {tuple(k):v for k,v in zip(H.terms, H.weights)}
shots= 100
# Run Tabu Search
final_spins, Nevals = tabu_search(hamiltonian, nq, max_iterations=p)
print("Final Spin Configuration:", final_spins)
spin = {"0":-1, "1":1}
print("Minimum Energy:", compute_energy({i:spin[sol[i]] for i in range(nq)}, hamiltonian))
print("Nevals:", Nevals)



Final Spin Configuration: {0: -1, 1: -1, 2: -1, 3: 1, 4: -1, 5: -1, 6: 1, 7: -1, 8: 1, 9: 1, 10: 1, 11: -1, 12: 1, 13: -1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: -1, 21: -1, 22: -1, 23: 1, 24: -1, 25: -1, 26: 1, 27: 1, 28: 1, 29: -1, 30: 1, 31: 1, 32: -1, 33: -1, 34: -1, 35: 1, 36: -1, 37: -1, 38: -1, 39: 1}
Minimum Energy: -29.167999999999974
Nevals: 201


In [6]:
problems = np.load(f"./Data/WMaxCut_cte_no_mdl.npy", allow_pickle=True).item()
results_sa = np.load(f"./Data/WMC_SA_results.npy", allow_pickle=True).item()

In [138]:
shots = 1_000
results = np.load("./Data/WMC_TABU_results.npy", allow_pickle=True).item()
# nqs = [40]
# nqs = [10,16,20,24,30,34]
arg_sort_dict = np.load("./Data/hard_FC_WMC.npy", allow_pickle=True).item()

nqs = [20,25,30,35,40]
ps = [200]
for nodes in nqs: 
    arg_sort = arg_sort_dict[nodes]

    print(f"number of nodes {nodes}")
    # results[nodes] = {}
    for kk in problems[nodes].keys():#arg_sort:
        if not kk % 10:
            print(f"----- k: {kk} ------")
        # results[nodes][kk] = defaultdict(dict)
        G = problems[nodes][kk]["G"]
        H = problems[nodes][kk]["ising_hamiltonian"]
        hamiltonian = {tuple(sorted(k)):v for k,v in zip(H.terms, H.weights) if abs(v) > 1e-10}
        # results[nodes][kk]["sol"] = results_sa[nodes][kk]["sol"]
        for p in ps:
            # results[nodes][kk]["TABU"][p] = customized_running_tabu(hamiltonian, shots, results[nodes][kk]["sol"], max_iterations=p, tabu_size=10)
            results[nodes][kk]["TABU"][p] = running_tabu(hamiltonian, shots, results[nodes][kk]["sol"], p)


number of nodes 20
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 25
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 30
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 35
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 40
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 -----

In [139]:
np.save(f"./Data/WMC_TABU_results.npy", results)